In [24]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [25]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7, max_num_hands = 2)

In [26]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [27]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

In [28]:
def extract_keypoints(results):
    data_aux = []

    x_ = []
    y_ = []

    if results.multi_hand_landmarks:
        
        if len(results.multi_hand_landmarks)==1:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

                for i in range(42, 84):
                    data_aux.append(0.0)

        if len(results.multi_hand_landmarks)==2:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_)) 
                
                
    else:
        for i in range(84):
            data_aux.append(0.0)                
    return data_aux  
 

In [31]:
DATA_DIR = 'C:/Users/Turbo/Desktop/char_data'
actions = []
for dir_ in os.listdir(DATA_DIR):
    actions.append(dir_)
actions

['Ain',
 'Alef',
 'Beh',
 'Dad',
 'Dal',
 'Feh',
 'Ghain',
 'Hah',
 'Heh',
 'Jeem',
 'Kaf',
 'Khah',
 'Lam',
 'Meem',
 'Noon',
 'Qaf',
 'Reh',
 'Sad',
 'Seen',
 'Sheen',
 'Tah',
 'Teh',
 'Thal',
 'Theh',
 'Waw',
 'Yeh',
 'Zah',
 'Zain']

In [32]:
from keras.models import load_model

# Path to your saved model file (.h5 format)

# Load the model
modelLSTM =  load_model("action2.h5")

In [34]:
hands = mp_hands.Hands(min_tracking_confidence=0.7, min_detection_confidence=0.7 , max_num_hands = 2, static_image_mode=True )

sentence = []

cap = cv2.VideoCapture(0)


while cap.isOpened():

    
    ret, frame = cap.read()

    image, results = mediapipe_detection(frame, hands)

    if results.multi_hand_landmarks:
        keypoints = extract_keypoints(results)
        draw_styled_landmarks(frame, results)

        if len(results.multi_hand_landmarks) >= 2:
        # Iterate through each detected hand
            
      

            if hasattr(model, "predict_proba"):
                keypoints1=keypoints[:42]
                predicted_probabilities = model.predict_proba([np.asarray(keypoints1)])
                predicted_class = np.argmax(predicted_probabilities)
                confidence1 = predicted_probabilities[0, predicted_class]


            
            if hasattr(model, "predict_proba"):
                keypoints2=keypoints[42:84]
                predicted_probabilities = model.predict_proba([np.asarray(keypoints2)])
                predicted_class = np.argmax(predicted_probabilities)
                confidence2 = predicted_probabilities[0, predicted_class]


            if confidence1 > confidence2:
                high_confidence_data = keypoints1
            else:
                high_confidence_data = keypoints2
            
            print("Confidence:", max(confidence1,confidence2))    
            prediction = model.predict([np.asarray(high_confidence_data)])
            predicted_character = label_map[int(prediction[0])]
            print(predicted_character)

            if (max(confidence1,confidence2) > 0.7):
                if len(sentence) > 0: 
                    if predicted_character != sentence[-1]:
                        sentence.append(predicted_character)
                else:
                    sentence.append(predicted_character)
            if len(sentence) > 3: 
                sentence = sentence[-3:]
            

            
        elif len(results.multi_hand_landmarks) == 1:
            

    #         if(len(data_aux) < 85):
            if hasattr(model, "predict_proba"):
                predicted_probabilities = model.predict_proba([np.asarray(keypoints[:42])])
                predicted_class = np.argmax(predicted_probabilities)
                confidence = predicted_probabilities[0, predicted_class]
                print("Confidence:", confidence)
                prediction = model.predict([keypoints[:42]])
                predicted_character = label_map[int(prediction[0])]
                print(predicted_character)
                
                if (confidence > 0.7):
                    if len(sentence) > 0: 
                        if predicted_character != sentence[-1]:
                            sentence.append(predicted_character)
                    else:
                        sentence.append(predicted_character)
                if len(sentence) > 3: 
                    sentence = sentence[-3:]
                    


                
            
    cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(frame, ' '.join(sentence), (3,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)   

    cv2.imshow('frame', frame)


    if cv2.waitKey(10) & 0xFF == ord('q'):
         break
cap.release()
cv2.destroyAllWindows()

Confidence: 0.56
Kaf
Confidence: 0.65
Kaf
Confidence: 0.78
Kaf
Confidence: 0.75
Kaf
Confidence: 0.65
Kaf
Confidence: 0.7
Kaf
Confidence: 0.69
Kaf
Confidence: 0.7
Kaf
Confidence: 0.69
Kaf
Confidence: 0.68
Kaf
Confidence: 0.7
Kaf
Confidence: 0.71
Kaf
Confidence: 0.69
Kaf
Confidence: 0.67
Kaf
Confidence: 0.68
Kaf
Confidence: 0.69
Kaf
Confidence: 0.68
Kaf
Confidence: 0.68
Kaf
Confidence: 0.65
Kaf
Confidence: 0.64
Kaf
Confidence: 0.64
Kaf
Confidence: 0.67
Kaf
Confidence: 0.64
Kaf
Confidence: 0.62
Kaf
Confidence: 0.64
Kaf
Confidence: 0.7
Ghain
Confidence: 0.79
Alef
Confidence: 0.88
Alef
Confidence: 0.95
Alef
Confidence: 0.94
Alef
Confidence: 0.96
Alef
Confidence: 0.95
Alef
Confidence: 0.8
Ghain
Confidence: 0.7
Alef
Confidence: 0.73
Ghain
Confidence: 0.66
Kaf
Confidence: 0.66
Kaf
Confidence: 0.67
Kaf
Confidence: 0.68
Kaf
Confidence: 0.66
Kaf
Confidence: 0.67
Kaf
Confidence: 0.67
Kaf
Confidence: 0.63
Kaf
Confidence: 0.64
Kaf
Confidence: 0.64
Kaf
Confidence: 0.64
Kaf
Confidence: 0.66
Kaf
Confid

Hah
Confidence: 0.16
Hah
Confidence: 0.16
Hah
Confidence: 0.16
Hah
Confidence: 0.16
Khah
Confidence: 0.48
Noon
Confidence: 0.71
Meem
Confidence: 0.72
Meem
Confidence: 0.74
Meem
Confidence: 0.73
Meem
Confidence: 0.71
Meem
Confidence: 0.7
Meem
Confidence: 0.7
Meem
Confidence: 0.68
Meem
Confidence: 0.64
Meem
Confidence: 0.67
Meem
Confidence: 0.64
Meem
Confidence: 0.26
Meem
Confidence: 0.25
Sad
Confidence: 0.65
Meem
Confidence: 0.39
Meem
Confidence: 0.35
Meem
Confidence: 0.43
Meem
Confidence: 0.44
Meem
Confidence: 0.35
Meem
Confidence: 0.38
Seen
Confidence: 0.47
Seen
Confidence: 0.35
Seen
Confidence: 0.32
Seen
Confidence: 0.35
Seen
Confidence: 0.47
Seen
Confidence: 0.5
Seen
Confidence: 0.5
Seen
Confidence: 0.52
Seen
Confidence: 0.57
Seen
Confidence: 0.58
Seen
Confidence: 0.57
Seen
Confidence: 0.55
Seen
Confidence: 0.57
Seen
Confidence: 0.58
Seen
Confidence: 0.58
Seen
Confidence: 0.73
Meem
Confidence: 0.66
Meem
Confidence: 0.77
Meem
Confidence: 0.71
Meem
Confidence: 0.36
Sheen
Confidence: 0